## **Test your Wav2vec2 model with and without Language Model**
Apply the language model that you made earlier and check the output of the model with and without the language model, you will see that the language model improves the WER criterion but slightly lowers the transcription speed.

## With LM

In [ ]:
from pyctcdecode import build_ctcdecoder
from transformers import AutoProcessor
from transformers import Wav2Vec2ProcessorWithLM

model_id = "/home/sadeghi/wav2vec2/fa/checkpoint-57000"

# LM must be in arpa format (for now)
lm_path = "/home/sadeghi/wav2vec2/fa/language_model/13Aug2022/5gram.arpa"


#processor = AutoProcessor.from_pretrained(model_id)
vocab_dict = processor.tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    lm_path,
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

processor_with_lm.save_pretrained("/home/sadeghi/wav2vec2/fa/language_model/13Aug2022/processor/")

Loading the LM will be faster if you build a binary file.
Reading /home/sadeghi/wav2vec2/fa/language_model/13Aug2022/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [ ]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM

model_name_or_path = "/home/sadeghi/wav2vec2/fa/checkpoint-57000" #import model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #choose device type
print(model_name_or_path, device)


processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/sadeghi/wav2vec2/fa/language_model/13Aug2022/processor/") #load pretrained processor with LM
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path) #load finetuned model


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(
        batch["speech"], 
        sampling_rate=processor.feature_extractor.sampling_rate, 
        return_tensors="pt", 
        padding=True
    )

    input_values = features.input_values
    attention_mask = features.attention_mask

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits #when we are trying to load model with LM we have to use logits instead of argmax(logits)
 
    batch["predicted"] = processor.batch_decode(logits.numpy()).text
    return batch

/home/sadeghi/wav2vec2/fa/checkpoint-57000 cuda


Loading the LM will be faster if you build a binary file.
Reading /home/sadeghi/wav2vec2/fa/language_model/13Aug2022/processor/language_model/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [ ]:
import torchaudio
import librosa
from datasets import load_dataset
import numpy as np

dataset = load_dataset("csv", data_files={"test":"/home/sadeghi/wav2vec2/fa/test_data.csv"}, delimiter=",")["test"]

dataset = dataset.map(speech_file_to_array_fn)

In [ ]:
result = dataset.map(predict, batched=True, batch_size=4)

  0%|          | 0/79 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_metric
wer = load_metric("wer")
cer = load_metric("cer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["refrence"])))
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["predicted"], references=result["refrence"])))

In [ ]:
for i in range(len(result)):
    reference, predicted =  result["refrence"][i], result["predicted"][i]
    if reference.strip() == predicted.strip():
        continue
    print("reference:", reference)
    print("predicted:", predicted)
    print('---')


reference: ‌بیمه پاسارگاد نمایندگی خبانزاده ‌
predicted: بیمه پاساگارد نمایندگی خبانزاده
---
reference: ‌برعکس ‌
predicted: برعکس
---
reference: ‌‌ایی‌‌ان‌ من وام رو واسه خرید ماشین می خوام ‌ایی‌‌ان‌ ‌
predicted: من وام و واسه خرید ماشین می خوام
---
reference: ‌سلام حال شما چطوره ‌
predicted: سلام حال شما چه دارین
---
reference: ‌مصالح زهکش ‌
predicted: مساله ذهکش
---
reference: ‌جذب آب ‌
predicted: جزب آب
---
reference: ‌ویرگول ‌
predicted: ویر گود
---
reference: از جمله اینکه این نرم‌افزار‌ها به صورت شبانه روزی و خستگی ناپذیر در دسترس هستند 
predicted: از جمله این که این نرم‌‌افزار‌ها به صورت شبانه روزی و خستگی ناپذیر در دسترس هستند
---
reference: امروزه توسعه علوم پردازش گفتار پردازش زبان طبیعی و علوم مرتبط دیگر موجب برآورده شدن این هدف گردیده است 
predicted: امروزه توسعه علوم پردازش گفتار پردازش زبان طبیعی و علوم مرتبط دیگر موجب برآورده شدن این هدف کردیده است
---
reference: و تنظیمات صدای کامپیو‌تر خود را دستکاری نکنید 
predicted: و تنظیمات صدای کامپیوتر خود را دستکاری نکنید
---
re

reference: خدا می خواد ما با تو زندگی کنیم 
predicted: خدا میخواد ما با تو زندگی کنیم
---
reference: چرا اینقدر همه رو اذیت می کنن 
predicted: چنین این قدر همه را اذیت میکنن
---
reference: برای یادآوری دوباره دوست دارم خداحافظ 
predicted: برای یادآوری دوباره دوستت دارم خداحافظ
---
reference: نباید بهش اعتماد کنی و دخترت رو بدی دستش 
predicted: نباید بهش اعتماد کنیم و دخترت رو بگه دستش
---
reference: چطوری از انفجار نجات پیدا کرد 
predicted: چه طوری از انفجار نجات پیدا کرد
---
reference: تو هم کار معامله رو انجام میدی 
predicted: تو هم کار معامله را انجام میدی
---
reference: از نظر طبقه متوسط تقویت ارتش هدر دادن وحشتناک پولی بود 
predicted: از نظر ترقه متوسط تقویت ارزش حدر دادن وحشتناک پولی بود
---
reference: حتی واسه یه لحظه هم به تو مشکوک نشده بودم 
predicted: حتی واسید یه لحظه هم به تو مشکول نشده بودم
---
reference: اوه عجب بدبختی ای هستم 
predicted: اوه عجب بدبختی هستم
---
reference: می خواست از من دزدی کنه 
predicted: میخواست از من دزدی کنه ز
---
reference: شیخ جلوی ما را گرفت 
pre

reference: هیچ وقت تا اون موقع تو رو اونقدر عصبی ندیده بودم 
predicted: هیچ وقت آن موقع تو را انقدر عصبی ندیده بودم
---
reference: ممکنه کسی که باید پول رو برمیداشته دیر رسیده 
predicted: ممکنه کسی باید باید پول رو برمیداشت و دیر رسیده
---
reference: خوب من هم درگیر تیراندازی بودم 
predicted: خو من هم درگیر تیر اندازی بودم
---
reference: برای یکی از دوستات میخونم 
predicted: برای یکی از دوستات می خونم
---
reference: بیخیال منو اون کاملا متفاوتیم 
predicted: به خیال من و عمو اون کاملا متفاوتیم
---
reference: تو لایق بیش‌تر از این هستی 
predicted: تو دقیقه بیش‌تر از من هستی
---
reference: من به جرمی که انجام ندادم متهم شدم 
predicted: به جاری که انجام ندادم متحم شدم
---
reference: آدما خیلی زود گول میخورن 
predicted: آدم‌ها خیلی زودگور می خورن
---
reference: مدت کوتاهی در یکی از شهر‌های کوچک انگلستان خدمت کردم 
predicted: مدت کوتاهی در یکی از شهر‌های کوچک انگلستان خدمت کردن
---
reference: حالا بریم نمی خوایم دیر بشه 
predicted: حالا بریم نمی خوام دیر بشه
---
reference: اینجوری با یه خانم

In [ ]:
from datasets import load_metric
wer = load_metric("wer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["refrence"])))

## Without LM

In [ ]:
model_name_or_path = "/home/sadeghi/wav2vec2/fa/checkpoint-57000"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(
        batch["speech"], 
        sampling_rate=processor.feature_extractor.sampling_rate, 
        return_tensors="pt", 
        padding=True
    )

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits 

    pred_ids = torch.argmax(logits, dim=-1)

    batch["predicted_N_LM"] = processor.batch_decode(pred_ids)
    return batch



/home/sadeghi/wav2vec2/fa/checkpoint-57000 cuda


In [ ]:
result = dataset.map(predict, batched=True, batch_size=4)

  0%|          | 0/79 [00:00<?, ?ba/s]

In [ ]:
wer = load_metric("wer")
cer = load_metric("cer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted_N_LM"],
                                             references=result["refrence"])))
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["predicted_N_LM"],
                                             references=result["refrence"])))

In [ ]:
for i in range(len(result)):
    reference, predicted_N_LM =  result["refrence"][i], result["predicted_N_LM"][i]
    if reference.strip() == predicted_N_LM.strip():
        continue
    print("reference:", reference)
    print("predicted:", predicted_N_LM)
    print('---')
